In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,RMSprop, Ftrl
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D,Normalization,Discretization
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, Normalization
)
from keras.callbacks import EarlyStopping
from keras.metrics import RootMeanSquaredError
from keras.layers import Flatten, Dense, Reshape


In [2]:
Xl = pd.read_csv('Xl.csv',header=None)
Xs = pd.read_csv('Xs.csv',header=None)
R = pd.read_csv('y.csv',header = None)

In [3]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [4]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [5]:
np.shape(t_test_end)

(552,)

In [6]:
Xl.shape

(744, 137)

In [7]:
Xs.shape

(744, 9)

In [8]:
t_test_end[551]

731

# MODEL 20

In [ ]:
from keras.engine import input_layer
loss = []
preds = []
seed(60)
for retrain_idx in range(552):
    print('Iterations Left:', 552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model2 = Sequential()
    model2.add(Discretization())
    model2.add(Dense(64,input_dim=135))
    model2.add(BatchNormalization())
    model2.add(Dense(8))
    model2.add(Dense(1))
    opt=RMSprop(learning_rate=0.0008)
    model2.compile(optimizer=opt,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model2.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256,epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model2.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
Epoch 9/20
1/1 [==============================] - 0s 48ms/step - loss: 0.0196 - val_loss: 0.0177
Epoch 10/20
1/1 [==============================] - 0s 45ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 11/20
1/1 [==============================] - 0s 42ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 12/20
1/1 [==============================] - 0s 39ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 13/20
1/1 [==============================] - 0s 42ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 14/20
1/1 [==============================] - 0s 39ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 15/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0196 - val_loss: 0.0176
Epoch 16/20
1/1 [==============================] - 0s 49ms/step - loss: 0.0196 - val_loss: 0.0176
Iterations Left: 351
Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.0402 - val_loss: 0.0292
Epoch 2/20
1/1 [==============================] - 

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.116658
dtype: float64

# MODEL 21

In [ ]:
from keras.engine import input_layer
loss = []
preds = []
seed(60)
for retrain_idx in range(552):
    print('Iterations Left:', 552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model3 = Sequential()
    model3.add(Discretization())
    model3.add(Dense(64,input_dim=135))
    model3.add(BatchNormalization())
    model3.add(Dense(8))
    model3.add(Dense(1))
    opt=Adam(learning_rate=0.0001)
    model3.compile(optimizer=opt,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=2)
    history = model3.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256,epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model3.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 45ms/step - loss: 0.0427 - val_loss: 0.0289
Epoch 7/20
1/1 [==============================] - 0s 43ms/step - loss: 0.0424 - val_loss: 0.0288
Epoch 8/20
1/1 [==============================] - 0s 41ms/step - loss: 0.0421 - val_loss: 0.0287
Epoch 9/20
1/1 [==============================] - 0s 44ms/step - loss: 0.0419 - val_loss: 0.0285
Epoch 10/20
1/1 [==============================] - 0s 46ms/step - loss: 0.0416 - val_loss: 0.0284
Epoch 11/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0413 - val_loss: 0.0283
Epoch 12/20
1/1 [==============================] - 0s 55ms/step - loss: 0.0411 - val_loss: 0.0282
Epoch 13/20
1/1 [==============================] - 0s 41ms/step - loss: 0.0408 - val_loss: 0.0280
Epoch 14/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0406 - val_loss: 0.0279
Epoch 15/20
1/1 [==============================] - 0s 51ms/step - loss: 0.0403 - v

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.378634
dtype: float64

# MODEL 22

In [ ]:
from keras.engine import input_layer
loss = []
preds = []
seed(60)
for retrain_idx in range(552):
    print('Iterations Left:', 552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model3 = Sequential()
    model3.add(Discretization())
    model3.add(Dense(135,input_dim=135))
    model3.add(BatchNormalization())
    model3.add(Dense(45))
    model3.add(Dense(1))
    opt=Adam(learning_rate=0.0001)
    model3.compile(optimizer=opt,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=2)
    history = model3.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256,epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model3.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
Epoch 11/20
1/1 [==============================] - 0s 53ms/step - loss: 0.0360 - val_loss: 0.0187
Epoch 12/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0355 - val_loss: 0.0183
Epoch 13/20
1/1 [==============================] - 0s 43ms/step - loss: 0.0350 - val_loss: 0.0179
Epoch 14/20
1/1 [==============================] - 0s 37ms/step - loss: 0.0345 - val_loss: 0.0175
Epoch 15/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0340 - val_loss: 0.0172
Epoch 16/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0335 - val_loss: 0.0168
Epoch 17/20
1/1 [==============================] - 0s 44ms/step - loss: 0.0331 - val_loss: 0.0165
Epoch 18/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0327 - val_loss: 0.0162
Epoch 19/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0323 - val_loss: 0.0159
Epoch 20/20
1/1 [==============================] - 0s 40ms/step - l

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.267751
dtype: float64

# MODEL 23

In [ ]:
from keras.engine import input_layer
loss = []
preds = []
seed(60)
for retrain_idx in range(552):
    print('Iterations Left:', 552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model3 = Sequential()
    model3.add(Discretization())
    model3.add(Dense(135,input_dim=135))
    model3.add(Dense(45))
    model3.add(Dense(1))
    opt=Adam()
    model3.compile(optimizer=opt,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=2)
    history = model3.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256,epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model3.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 1s 705ms/step - loss: 0.0429 - val_loss: 0.0353
Epoch 2/20
1/1 [==============================] - 0s 42ms/step - loss: 0.0375 - val_loss: 0.0283
Epoch 3/20
1/1 [==============================] - 0s 50ms/step - loss: 0.0336 - val_loss: 0.0230
Epoch 4/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0316 - val_loss: 0.0197
Epoch 5/20
1/1 [==============================] - 0s 44ms/step - loss: 0.0314 - val_loss: 0.0184
Epoch 6/20
1/1 [==============================] - 0s 39ms/step - loss: 0.0325 - val_loss: 0.0181
Epoch 7/20
1/1 [==============================] - 0s 45ms/step - loss: 0.0336 - val_loss: 0.0181
Epoch 8/20
1/1 [==============================] - 0s 46ms/step - loss: 0.0338 - val_loss: 0.0182
Epoch 9/20
1/1 [==============================] - 0s 68ms/step - loss: 0.0333 - val_loss: 0.0184
Iterations Left: 380
Epoch 1/20
1/1 [==============================] - 1s 724ms/step -

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]


In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.13422
dtype: float64

# MODEL 24

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_10 = Sequential()
    model_10.add(Discretization())
    model_10.add(Dense(135))
    model_10.add(BatchNormalization())
    model_10.add(Dense(32))
    model_10.add(Dense(1))
    model_10.compile(optimizer='Adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_10.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=30,verbose=1,callbacks=[early_stop])
    preds.append(model_10.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
Iterations pendings: 421
Epoch 1/30
1/1 [==============================] - 1s 775ms/step - loss: 0.0304 - val_loss: 0.0442
Epoch 2/30
1/1 [==============================] - 0s 40ms/step - loss: 0.0273 - val_loss: 0.0388
Epoch 3/30
1/1 [==============================] - 0s 38ms/step - loss: 0.0250 - val_loss: 0.0342
Epoch 4/30
1/1 [==============================] - 0s 41ms/step - loss: 0.0236 - val_loss: 0.0306
Epoch 5/30
1/1 [==============================] - 0s 39ms/step - loss: 0.0232 - val_loss: 0.0282
Epoch 6/30
1/1 [==============================] - 0s 33ms/step - loss: 0.0237 - val_loss: 0.0271
Epoch 7/30
1/1 [==============================] - 0s 44ms/step - loss: 0.0243 - val_loss: 0.0268
Epoch 8/30
1/1 [==============================] - 0s 39ms/step - loss: 0.0247 - val_loss: 0.0268
Iterations pendings: 420
Epoch 1/30
1/1 [==============================] - 1s 787ms/step - loss: 0.0314 - val_loss: 0.0430
Epoch 2/30
1/1 [========

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]


In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.094135
dtype: float64

# MODEL 25

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_10 = Sequential()
    model_10.add(Discretization())
    model_10.add(Dense(135))
    model_10.add(BatchNormalization())
    model_10.add(Dense(32))
    model_10.add(Dense(16))
    model_10.add(Dense(1))
    model_10.compile(optimizer='Adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_10.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=30,verbose=1,callbacks=[early_stop])
    preds.append(model_10.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 34ms/step - loss: 0.0212 - val_loss: 0.0177
Epoch 3/30
1/1 [==============================] - 0s 36ms/step - loss: 0.0214 - val_loss: 0.0167
Epoch 4/30
1/1 [==============================] - 0s 31ms/step - loss: 0.0216 - val_loss: 0.0173
Iterations pendings: 486
Epoch 1/30
1/1 [==============================] - 1s 812ms/step - loss: 0.0223 - val_loss: 0.0213
Epoch 2/30
1/1 [==============================] - 0s 34ms/step - loss: 0.0211 - val_loss: 0.0174
Epoch 3/30
1/1 [==============================] - 0s 30ms/step - loss: 0.0215 - val_loss: 0.0167
Epoch 4/30
1/1 [==============================] - 0s 31ms/step - loss: 0.0217 - val_loss: 0.0176
Iterations pendings: 485
Epoch 1/30
1/1 [==============================] - 1s 740ms/step - loss: 0.0225 - val_loss: 0.0209
Epoch 2/30
1/1 [==============================] - 0s 33ms/step - loss: 0.0213 - val_loss: 0.0170
Epoch 3/30
1/1 [===================

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]


In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.091402
dtype: float64

# MODEL 26

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_10 = Sequential()
    model_10.add(Discretization())
    model_10.add(Dense(135))
    model_10.add(BatchNormalization())
    model_10.add(Dense(64))
    model_10.add(BatchNormalization())
    model_10.add(Dense(32))
    model_10.add(Dense(1))
    model_10.compile(optimizer='Adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_10.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=30,verbose=1,callbacks=[early_stop])
    preds.append(model_10.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
Epoch 5/30
1/1 [==============================] - 0s 30ms/step - loss: 0.0232 - val_loss: 0.0226
Epoch 6/30
1/1 [==============================] - 0s 32ms/step - loss: 0.0211 - val_loss: 0.0196
Epoch 7/30
1/1 [==============================] - 0s 31ms/step - loss: 0.0199 - val_loss: 0.0177
Epoch 8/30
1/1 [==============================] - 0s 29ms/step - loss: 0.0196 - val_loss: 0.0168
Epoch 9/30
1/1 [==============================] - 0s 30ms/step - loss: 0.0201 - val_loss: 0.0166
Epoch 10/30
1/1 [==============================] - 0s 30ms/step - loss: 0.0210 - val_loss: 0.0167
Iterations pendings: 363
Epoch 1/30
1/1 [==============================] - 1s 911ms/step - loss: 0.0392 - val_loss: 0.0433
Epoch 2/30
1/1 [==============================] - 0s 45ms/step - loss: 0.0341 - val_loss: 0.0372
Epoch 3/30
1/1 [==============================] - 0s 42ms/step - loss: 0.0297 - val_loss: 0.0319
Epoch 4/30
1/1 [==============================] -

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]



In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.097769
dtype: float64

# MODEL 27

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_10 = Sequential()
    model_10.add(Discretization())
    model_10.add(Dense(135))
    model_10.add(BatchNormalization())
    model_10.add(Dense(64))
    model_10.add(BatchNormalization())
    model_10.add(Dense(32))
    model_10.add(BatchNormalization())
    model_10.add(Dense(16))
    model_10.add(Dense(1))
    model_10.compile(optimizer='Adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_10.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model_10.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 38ms/step - loss: 0.0421 - val_loss: 0.0205
Epoch 3/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0383 - val_loss: 0.0168
Epoch 4/20
1/1 [==============================] - 0s 43ms/step - loss: 0.0348 - val_loss: 0.0134
Epoch 5/20
1/1 [==============================] - 0s 39ms/step - loss: 0.0317 - val_loss: 0.0107
Epoch 6/20
1/1 [==============================] - 0s 37ms/step - loss: 0.0290 - val_loss: 0.0089
Epoch 7/20
1/1 [==============================] - 0s 31ms/step - loss: 0.0267 - val_loss: 0.0080
Epoch 8/20
1/1 [==============================] - 0s 32ms/step - loss: 0.0249 - val_loss: 0.0079
Epoch 9/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0235 - val_loss: 0.0086
Iterations pendings: 290
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0470 - val_loss: 0.0272
Epoch 2/20
1/1 [==============================] - 0s 55ms/step -

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.079603
dtype: float64

# MODEL 28

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_11 = Sequential()
    model_11.add(Discretization())
    model_11.add(Dense(135))
    model_11.add(BatchNormalization())
    model_11.add(Dense(64))
    model_11.add(BatchNormalization())
    model_11.add(Dense(32))
    model_11.add(BatchNormalization())
    model_11.add(Dense(16))
    model_11.add(Dense(1))
    model_11.compile(optimizer='RMSProp',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_11.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model_11.predict(X_test))
    loss.append(min(history.history['val_loss']))

Iterations pendings: 552
Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 0.0211 - val_loss: 0.0363
Epoch 2/20
1/1 [==============================] - 0s 34ms/step - loss: 0.0157 - val_loss: 0.0396
Iterations pendings: 551
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0211 - val_loss: 0.0319
Epoch 2/20
1/1 [==============================] - 0s 33ms/step - loss: 0.0154 - val_loss: 0.0328
Iterations pendings: 550
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0212 - val_loss: 0.0326
Epoch 2/20
1/1 [==============================] - 0s 34ms/step - loss: 0.0149 - val_loss: 0.0383
Iterations pendings: 549
Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.0212 - val_loss: 0.0318
Epoch 2/20
1/1 [==============================] - 0s 32ms/step - loss: 0.0158 - val_loss: 0.0357
Iterations pendings: 548
Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.0212 - val_loss: 0.0318
Epoch 2/20
1

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]


In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.190148
dtype: float64

# MODEL 29

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_11 = Sequential()
    model_11.add(Discretization())
    model_11.add(Dense(135))
    model_11.add(BatchNormalization())
    model_11.add(Dense(64))
    model_11.add(BatchNormalization())
    model_11.add(Dense(32))
    model_11.add(BatchNormalization())
    model_11.add(Dense(16))
    model_11.add(Dense(1))
    model_11.compile(optimizer='nadam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_11.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=128, epochs=20,verbose=1,callbacks=[early_stop])
    preds.append(model_11.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 43ms/step - loss: 0.0282 - val_loss: 0.0198
Epoch 8/20
1/1 [==============================] - 0s 40ms/step - loss: 0.0264 - val_loss: 0.0180
Epoch 9/20
1/1 [==============================] - 0s 38ms/step - loss: 0.0248 - val_loss: 0.0185
Iterations pendings: 340
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0452 - val_loss: 0.0244
Epoch 2/20
1/1 [==============================] - 0s 62ms/step - loss: 0.0417 - val_loss: 0.0236
Epoch 3/20
1/1 [==============================] - 0s 49ms/step - loss: 0.0393 - val_loss: 0.0225
Epoch 4/20
1/1 [==============================] - 0s 49ms/step - loss: 0.0372 - val_loss: 0.0227
Iterations pendings: 339
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.0460 - val_loss: 0.0260
Epoch 2/20
1/1 [==============================] - 0s 47ms/step - loss: 0.0425 - val_loss: 0.0233
Epoch 3/20
1/1 [=========================

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.242925
dtype: float64

# MODEL 30

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_11 = Sequential()
    model_11.add(Discretization())
    model_11.add(Dense(135))
    model_11.add(BatchNormalization())
    model_11.add(Dense(64))
    model_11.add(BatchNormalization())
    model_11.add(Dense(32))
    model_11.add(BatchNormalization())
    model_11.add(Dense(16))
    model_11.add(Dense(1))
    model_11.compile(optimizer='adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
    history = model_11.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256, epochs=30,verbose=1,callbacks=[early_stop])
    preds.append(model_11.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 41ms/step - loss: 0.0226 - val_loss: 0.0088
Epoch 13/30
1/1 [==============================] - 0s 39ms/step - loss: 0.0223 - val_loss: 0.0084
Epoch 14/30
1/1 [==============================] - 0s 36ms/step - loss: 0.0222 - val_loss: 0.0081
Epoch 15/30
1/1 [==============================] - 0s 58ms/step - loss: 0.0223 - val_loss: 0.0080
Epoch 16/30
1/1 [==============================] - 0s 47ms/step - loss: 0.0226 - val_loss: 0.0080
Epoch 17/30
1/1 [==============================] - 0s 45ms/step - loss: 0.0230 - val_loss: 0.0080
Epoch 18/30
1/1 [==============================] - 0s 38ms/step - loss: 0.0234 - val_loss: 0.0080
Iterations pendings: 294
Epoch 1/30
1/1 [==============================] - 1s 1s/step - loss: 0.0461 - val_loss: 0.0262
Epoch 2/30
1/1 [==============================] - 0s 60ms/step - loss: 0.0422 - val_loss: 0.0224
Epoch 3/30
1/1 [==============================] - 0s 44ms/

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.10919
dtype: float64

# MODEL 31

In [ ]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    print('Iterations pendings:' ,552-retrain_idx)
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model_11 = Sequential()
    model_11.add(Discretization())
    model_11.add(Dense(135))
    model_11.add(BatchNormalization())
    model_11.add(Dense(64))
    model_11.add(BatchNormalization())
    model_11.add(Dense(32))
    model_11.add(BatchNormalization())
    model_11.add(Dense(16))
    model_11.add(Dense(1))
    model_11.compile(optimizer='adam',loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=3)
    history = model_11.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256, epochs=30,verbose=1,callbacks=[early_stop])
    preds.append(model_11.predict(X_test))
    loss.append(min(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 30ms/step - loss: 0.0141 - val_loss: 0.0107
Epoch 20/30
1/1 [==============================] - 0s 35ms/step - loss: 0.0144 - val_loss: 0.0109
Iterations pendings: 258
Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 0.0364 - val_loss: 0.0531
Epoch 2/30
1/1 [==============================] - 0s 45ms/step - loss: 0.0321 - val_loss: 0.0478
Epoch 3/30
1/1 [==============================] - 0s 43ms/step - loss: 0.0283 - val_loss: 0.0432
Epoch 4/30
1/1 [==============================] - 0s 44ms/step - loss: 0.0248 - val_loss: 0.0386
Epoch 5/30
1/1 [==============================] - 0s 35ms/step - loss: 0.0217 - val_loss: 0.0340
Epoch 6/30
1/1 [==============================] - 0s 40ms/step - loss: 0.0191 - val_loss: 0.0297
Epoch 7/30
1/1 [==============================] - 0s 40ms/step - loss: 0.0169 - val_loss: 0.0254
Epoch 8/30
1/1 [==============================] - 0s 42ms/step 

In [ ]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

In [ ]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.12949
dtype: float64